In [3]:
Nz = 120

I3_lte_m = np.zeros((5, Nz))
I3_nlt_m = np.zeros((5, Nz))
Is_lte_m = np.zeros((5, Nz))
Is_nlt_m = np.zeros((5, Nz))

mu = ['10', '8', '6', '4', '2']

for i in range(5):

    I3_lte_m[i, :] = np.load('./mean_spec/G2V.300G.627321.1.lte.' + mu[i] + '.npz')['I']
    I3_nlt_m[i, :] = np.load('./mean_spec/G2V.300G.627321.1.nlte.' + mu[i] + '.npz')['I']
    Is_lte_m[i, :] = np.load('./mean_spec/G2V.ssd.222809.1.lte.' + mu[i] + '.npz')['I']
    Is_nlt_m[i, :] = np.load('./mean_spec/G2V.ssd.222809.1.nlte.' + mu[i] + '.npz')['I']

In [1]:
import numpy as np

from tqdm import tqdm

import itertools

def fill_zeros(I, idx1, idx2):
    
    Izf = I
    
    for i in tqdm(range(Nx), desc = str(idx1) + ', ' + str(idx2)):
    
        for j in range(Ny):
            
            if Izf[i, j, 0] == 0.0:
                
                if i != 0:
                    
                    Izf[i, j, :] = Izf[i - 1, j, :]
                    
                else:
                    
                    Izf[i, j, :] = Izf[i, j - 1, :]
    
    return Izf

Nx = 512
Ny = 512
Nz = 120

I3_lte = np.zeros((5, Nx, Ny, Nz))
I3_nlt = np.zeros((5, Nx, Ny, Nz))
Is_lte = np.zeros((5, Nx, Ny, Nz))
Is_nlt = np.zeros((5, Nx, Ny, Nz))

mu = ['10', '8', '6', '4', '2']

for i in range(5):
    
    I3_lte[i, :, :, :] = fill_zeros(np.load('./spec/G2V.300G.627321.1.lte.' + mu[i] + '.npz')['I'], i, 1)
    I3_nlt[i, :, :, :] = fill_zeros(np.load('./spec/G2V.300G.627321.1.nlte.' + mu[i] + '.npz')['I'], i, 2)
    Is_lte[i, :, :, :] = fill_zeros(np.load('./spec/G2V.ssd.222809.1.lte.' + mu[i] + '.npz')['I'], i, 3)
    Is_nlt[i, :, :, :] = fill_zeros(np.load('./spec/G2V.ssd.222809.1.nlte.' + mu[i] + '.npz')['I'], i, 4)

#    for x, y in itertools.product(range(Nx), range(Ny)):

#        I3_lte[i, x, y, :] /= I3_lte_m[i, :]
#        I3_nlt[i, x, y, :] /= I3_nlt_m[i, :]
#        Is_lte[i, x, y, :] /= Is_lte_m[i, :]
#        Is_nlt[i, x, y, :] /= Is_nlt_m[i, :]

4, 4: 100%|██████████| 512/512 [00:00<00:00, 990.81it/s] 


In [8]:
Is_rat = np.zeros((5, Nx, Ny, Nz))
I3_rat = np.zeros((5, Nx, Ny, Nz))

Is_rat = Is_nlt / Is_lte
I3_rat = I3_nlt / I3_lte

lp = 0.1
up = 99.9

print('Calculating percentiles for ratios...')
        
smin = np.percentile(Is_rat, lp)
mmin = np.percentile(I3_rat, lp)
    
smax = np.percentile(Is_rat, up)
mmax = np.percentile(I3_rat, up)
    
rmin = min(smin, mmin)
rmax = max(smax, mmax)

print('Calculating percentiles for LTE...')

lmin = min(np.percentile(Is_lte, lp), np.percentile(I3_lte, lp))
lmax = max(np.percentile(Is_lte, up), np.percentile(I3_lte, up))

print('Calculating percentiles for NLTE...')
    
nmin = min(np.percentile(Is_nlt, lp), np.percentile(I3_nlt, lp))
nmax = max(np.percentile(Is_nlt, up), np.percentile(I3_nlt, up))
    
imin = min(lmin, nmin)
imax = max(lmax, nmax)

np.savez('min_max_norm', imin = imin, imax = imax, rmin = rmin, rmax = rmax)

Calculating percentiles for ratios...
Calculating percentiles for LTE...
Calculating percentiles for NLTE...


In [21]:
import matplotlib.pyplot as plt

import matplotlib.gridspec as gridspec
from matplotlib.colorbar import Colorbar

from matplotlib.ticker import AutoMinorLocator
from matplotlib.ticker import MultipleLocator

from matplotlib import colors

import os

from tqdm import tqdm

plt.rcParams.update({'font.size': 8})
plt.rcParams["font.family"] = 'Times New Roman'

Is_rat = np.zeros((5, Nx, Ny))
I3_rat = np.zeros((5, Nx, Ny))

w = np.arange(180, 300, 1) + 0.5

#j = 20

pdfs = ''

#for j in tqdm(range(Nz)):
for j in tqdm(range(19, 20)):
    
    pdfs += './fig/img/' + str(j) + '.pdf '

    plt.close('all')

    fig = plt.figure(dpi = 300)

    fig.set_size_inches(3, 10, forward = True)

    gs = gridspec.GridSpec(7, 2, height_ratios = [0.05, 1, 1, 1, 1, 1, 1])

    intensity_cbar = plt.subplot(gs[0, :])

#    colormap = 'hsv'
#    colormap = 'viridis'
    colormap = 'plasma'

    mus = ['1.0', '0.8', '0.6', '0.4', '0.2']
    
    for i in range(1, 6):

        Is_rat[i - 1, :, :] = Is_nlt[i - 1, :, :, j] / Is_lte[i - 1, :, :, j]
        I3_rat[i - 1, :, :] = I3_nlt[i - 1, :, :, j] / I3_lte[i - 1, :, :, j]
        
    smin = np.percentile(Is_rat, 5)
    mmin = np.percentile(I3_rat, 5)
    
    smax = np.percentile(Is_rat, 95)
    mmax = np.percentile(I3_rat, 95)
    
    gmin = min(smin, mmin)
    gmax = max(smax, mmax)

    for i in range(1, 6):

        ssd = plt.subplot(gs[i, 0])
        mag = plt.subplot(gs[i, 1])

        ssd.set_xlim(0, 511)
        ssd.set_ylim(0, 511)

        mag.set_xlim(0, 511)
        mag.set_ylim(0, 511)
    
        ssd.xaxis.set_ticklabels([])
        ssd.yaxis.set_ticklabels([])
    
        ssd.xaxis.set_ticks([])
        ssd.yaxis.set_ticks([])
    
        mag.xaxis.set_ticklabels([])
        mag.yaxis.set_ticklabels([])
    
        mag.xaxis.set_ticks([])
        mag.yaxis.set_ticks([])
    
        ssd.set_ylabel(r'$\mu = $' + mus[i - 1])
    
        ssd_img = ssd.imshow(Is_rat[i - 1, :, :], cmap = colormap, vmin = gmin, vmax = gmax, norm=colors.LogNorm())
        mag_img = mag.imshow(I3_rat[i - 1, :, :], cmap = colormap, vmin = gmin, vmax = gmax, norm=colors.LogNorm())
    
        cb = Colorbar(ax = intensity_cbar, mappable = ssd_img, orientation = 'horizontal', ticklocation = 'top')
    
        if i == 1:
        
            ssd.set_title('SSD')
            mag.set_title('300G')
        
#    cb.set_label(r'$I_\mathrm{NLTE}\bar{I}_\mathrm{LTE} / I_\mathrm{LTE} / \bar{I}_\mathrm{NLTE}$', labelpad = 5)
    cb.set_label(r'$I_\mathrm{NLTE} / I_\mathrm{LTE}$', labelpad = 5)
    
    spec = plt.subplot(gs[6, :])

    spec.plot(w, I3_lte_m[0, :], color = 'k', linewidth = 0.5, label = 'LTE')
    spec.plot(w, I3_nlt_m[0, :], color = 'r', linewidth = 0.5, label = 'NLTE')

    spec.axvline(x = w[j], linestyle = '--', linewidth = 0.5)

    spec.set_yscale('log')

    spec.set_xlim(180, 300)

    spec.set_xlabel('Wavelength, nm')
    spec.set_ylabel('Intensity, cgs')

    spec.xaxis.set_minor_locator(AutoMinorLocator(10))
    
    leg = spec.legend(framealpha = 1, loc = 4, handletextpad = 1, prop = {'size': 8.5})

#    plt.show()

    plt.savefig('./fig/img/' + str(j) + '.pdf', bbox_inches = 'tight')
    
os.system('pdftk ' + pdfs + ' output ./fig/img/all.pdf')

100%|██████████| 1/1 [00:44<00:00, 44.12s/it]


0

In [68]:
# this cell plots images only; no spectrum

import matplotlib.pyplot as plt

import matplotlib.gridspec as gridspec
from matplotlib.colorbar import Colorbar

from matplotlib.ticker import AutoMinorLocator
from matplotlib.ticker import MultipleLocator

from matplotlib import colors

import os

from tqdm import tqdm

plt.rcParams.update({'font.size': 8})
plt.rcParams["font.family"] = 'Times New Roman'

Is_rat = np.zeros((5, Nx, Ny))
I3_rat = np.zeros((5, Nx, Ny))

w = np.arange(180, 300, 1) + 0.5

#j = 20

pdfs = ''

#for j in tqdm(range(Nz)):
for j in tqdm(range(19, 20)):
    
    pdfs += './fig/img/' + str(j) + '.pdf '

    plt.close('all')

    fig = plt.figure(dpi = 300)

    fig.set_size_inches(5, 3, forward = True)

    fig.tight_layout()

    gs = gridspec.GridSpec(2, 6, width_ratios = [0.05, 1, 0.8, 0.6, 0.4, 0.2], hspace = 0.04, wspace = 0.06)

    intensity_cbar = plt.subplot(gs[:, 0])

#    colormap = 'hsv'
#    colormap = 'viridis'
    colormap = 'plasma'

    mus = ['1.0', '0.8', '0.6', '0.4', '0.2']
    
    for i in range(1, 6):

        Is_rat[i - 1, :, :] = Is_nlt[i - 1, :, :, j] / Is_lte[i - 1, :, :, j]
        I3_rat[i - 1, :, :] = I3_nlt[i - 1, :, :, j] / I3_lte[i - 1, :, :, j]
        
    smin = np.percentile(Is_rat, 5)
    mmin = np.percentile(I3_rat, 5)
    
    smax = np.percentile(Is_rat, 95)
    mmax = np.percentile(I3_rat, 95)
    
    gmin = min(smin, mmin)
    gmax = max(smax, mmax)

    for i in range(1, 6):

        ssd = plt.subplot(gs[0, i])
        mag = plt.subplot(gs[1, i])

        ssd.set_xlim(0, 511)
        ssd.set_ylim(0, 511)

        mag.set_xlim(0, 511)
        mag.set_ylim(0, 511)

        ssd.xaxis.set_ticklabels([])
        ssd.yaxis.set_ticklabels([])
    
        ssd.xaxis.set_ticks([])
        ssd.yaxis.set_ticks([])
    
        mag.xaxis.set_ticklabels([])
        mag.yaxis.set_ticklabels([])
    
        mag.xaxis.set_ticks([])
        mag.yaxis.set_ticks([])
    
        ssd_img = ssd.imshow(Is_rat[i - 1, :, :], cmap = colormap, vmin = gmin, vmax = gmax, norm = colors.LogNorm(), aspect = 'auto')
        mag_img = mag.imshow(I3_rat[i - 1, :, :], cmap = colormap, vmin = gmin, vmax = gmax, norm = colors.LogNorm(), aspect = 'auto')
    
        cb = Colorbar(ax = intensity_cbar, mappable = ssd_img, orientation = 'vertical', ticklocation = 'left')
        
        ssd.set_title(r'$\mu = $' + mus[i - 1], fontsize = 5)
    
        if i == 5:
            
            ssd.set_ylabel('SSD')
            mag.set_ylabel('300G')
            
            ssd.yaxis.set_label_position("right")
            mag.yaxis.set_label_position("right")
        
#    cb.set_label(r'$I_\mathrm{NLTE}\bar{I}_\mathrm{LTE} / I_\mathrm{LTE} / \bar{I}_\mathrm{NLTE}$', labelpad = 5)
    cb.set_label(r'$I_\mathrm{NLTE} / I_\mathrm{LTE}$', labelpad = 5)
    
#    spec = plt.subplot(gs[6, :])

#    spec.plot(w, I3_lte_m[0, :], color = 'k', linewidth = 0.5, label = 'LTE')
#    spec.plot(w, I3_nlt_m[0, :], color = 'r', linewidth = 0.5, label = 'NLTE')

#    spec.axvline(x = w[j], linestyle = '--', linewidth = 0.5)

#    spec.set_yscale('log')

#    spec.set_xlim(180, 300)

#    spec.set_xlabel('Wavelength, nm')
#    spec.set_ylabel('Intensity, cgs')

#    spec.xaxis.set_minor_locator(AutoMinorLocator(10))
    
#    leg = spec.legend(framealpha = 1, loc = 4, handletextpad = 1, prop = {'size': 8.5})

#    plt.show()

    plt.savefig('./fig/img/' + str(j) + '.pdf', bbox_inches = 'tight')
    
os.system('pdftk ' + pdfs + ' output ./fig/img/all.pdf')

100%|██████████| 1/1 [00:56<00:00, 56.90s/it]


0

In [71]:
#I_nlte / I_lte for SSD and 300G plus specta

import matplotlib.pyplot as plt

import matplotlib.gridspec as gridspec
from matplotlib.colorbar import Colorbar

from matplotlib.ticker import AutoMinorLocator
from matplotlib.ticker import MultipleLocator

from matplotlib import colors

import os

from tqdm import tqdm

plt.rcParams.update({'font.size': 8})
plt.rcParams["font.family"] = 'Times New Roman'

Is_rat = np.zeros((5, Nx, Ny))
I3_rat = np.zeros((5, Nx, Ny))

w = np.arange(180, 300, 1) + 0.5

#j = 20

pdfs = ''

for j in tqdm(range(Nz)):
#for j in tqdm(range(19, 20)):
    
    pdfs += './fig/img/' + str(j) + '.pdf '

    plt.close('all')

    fig = plt.figure(dpi = 300)

    fig.set_size_inches(5, 4.5, forward = True)

    fig.tight_layout()

    gs = gridspec.GridSpec(3, 6, width_ratios = [0.05, 1, 0.8, 0.6, 0.4, 0.2], hspace = 0.04, wspace = 0.06)

    intensity_cbar = plt.subplot(gs[:2, 0])

#    colormap = 'hsv'
#    colormap = 'viridis'
    colormap = 'plasma'

    mus = ['1.0', '0.8', '0.6', '0.4', '0.2']
    
    for i in range(1, 6):

        Is_rat[i - 1, :, :] = Is_nlt[i - 1, :, :, j] / Is_lte[i - 1, :, :, j]
        I3_rat[i - 1, :, :] = I3_nlt[i - 1, :, :, j] / I3_lte[i - 1, :, :, j]
        
    smin = np.percentile(Is_rat, 5)
    mmin = np.percentile(I3_rat, 5)
    
    smax = np.percentile(Is_rat, 95)
    mmax = np.percentile(I3_rat, 95)
    
    gmin = min(smin, mmin)
    gmax = max(smax, mmax)

    for i in range(1, 6):

        ssd = plt.subplot(gs[0, i])
        mag = plt.subplot(gs[1, i])

        ssd.set_xlim(0, 511)
        ssd.set_ylim(0, 511)

        mag.set_xlim(0, 511)
        mag.set_ylim(0, 511)

        ssd.xaxis.set_ticklabels([])
        ssd.yaxis.set_ticklabels([])
    
        ssd.xaxis.set_ticks([])
        ssd.yaxis.set_ticks([])
    
        mag.xaxis.set_ticklabels([])
        mag.yaxis.set_ticklabels([])
    
        mag.xaxis.set_ticks([])
        mag.yaxis.set_ticks([])
    
        ssd_img = ssd.imshow(Is_rat[i - 1, :, :], cmap = colormap, vmin = gmin, vmax = gmax, norm = colors.LogNorm(), aspect = 'auto')
        mag_img = mag.imshow(I3_rat[i - 1, :, :], cmap = colormap, vmin = gmin, vmax = gmax, norm = colors.LogNorm(), aspect = 'auto')
    
        cb = Colorbar(ax = intensity_cbar, mappable = ssd_img, orientation = 'vertical', ticklocation = 'left')
        
        ssd.set_title(r'$\mu = $' + mus[i - 1], fontsize = 5)
    
        if i == 5:
            
            ssd.set_ylabel('SSD')
            mag.set_ylabel('300G')
            
            ssd.yaxis.set_label_position("right")
            mag.yaxis.set_label_position("right")
        
#    cb.set_label(r'$I_\mathrm{NLTE}\bar{I}_\mathrm{LTE} / I_\mathrm{LTE} / \bar{I}_\mathrm{NLTE}$', labelpad = 5)
    cb.set_label(r'$I_\mathrm{NLTE} / I_\mathrm{LTE}$', labelpad = 5)
    
    spec = plt.subplot(gs[2, :])

    spec.plot(w, I3_lte_m[0, :], color = 'k', linewidth = 0.5, label = 'LTE')
    spec.plot(w, I3_nlt_m[0, :], color = 'r', linewidth = 0.5, label = 'NLTE')
    
    spec.axvline(x = w[j], linestyle = '--', linewidth = 0.5)

    spec.set_yscale('log')

    spec.set_xlim(180, 300)

    spec.set_xlabel('Wavelength, nm')
    spec.set_ylabel('Intensity, cgs')

    spec.xaxis.set_minor_locator(AutoMinorLocator(10))
    
    leg = spec.legend(framealpha = 1, loc = 4, handletextpad = 1, prop = {'size': 8.5})

#    plt.show()

    plt.savefig('./fig/img/' + str(j) + '.pdf', bbox_inches = 'tight')
    
os.system('pdftk ' + pdfs + ' output ./fig/img/all.pdf')

100%|██████████| 120/120 [09:31<00:00,  3.49s/it]


0

In [6]:
#I_nlte, I_lte and I_nlte / I_lte for SSD, plus specta

import matplotlib.pyplot as plt

import matplotlib.gridspec as gridspec
from matplotlib.colorbar import Colorbar

from matplotlib.ticker import AutoMinorLocator
from matplotlib.ticker import MultipleLocator

from matplotlib import colors

import os

from tqdm import tqdm

plt.rcParams.update({'font.size': 8})
plt.rcParams["font.family"] = 'Times New Roman'

Is_rat = np.zeros((5, Nx, Ny))

w = np.arange(180, 300, 1) + 0.5

#j = 20

pdfs = ''

#for j in tqdm(range(Nz)):
for j in tqdm(range(19, 20)):
    
    pdfs += './fig/img/' + str(j) + '.pdf '

    plt.close('all')

    fig = plt.figure(dpi = 300)

    fig.set_size_inches(5, 6, forward = True)

    fig.tight_layout()

    gs = gridspec.GridSpec(4, 6, width_ratios = [0.05, 1, 0.8, 0.6, 0.4, 0.2], hspace = 0.04, wspace = 0.06)

    int_cbar = plt.subplot(gs[:2, 0])
    rat_cbar = plt.subplot(gs[2, 0])

    colormap = 'plasma'

    mus = ['1.0', '0.8', '0.6', '0.4', '0.2']
    
    for i in range(1, 6):

        Is_rat[i - 1, :, :] = Is_nlt[i - 1, :, :, j] / Is_lte[i - 1, :, :, j]
        
    rmin = np.percentile(Is_rat, 5)
    rmax = np.percentile(Is_rat, 95)
    
    lmin = np.percentile(Is_lte[:, :, :, j], 5)
    lmax = np.percentile(Is_lte[:, :, :, j], 95)
    
    nmin = np.percentile(Is_nlt[:, :, :, j], 5)
    nmax = np.percentile(Is_nlt[:, :, :, j], 95)
    
    imin = min(lmin, nmin)
    imax = max(lmax, nmax)

    for i in range(1, 6):

        lte = plt.subplot(gs[0, i])
        nlt = plt.subplot(gs[1, i])
        
        rat = plt.subplot(gs[2, i])

        lte.set_xlim(0, 511)
        lte.set_ylim(0, 511)

        nlt.set_xlim(0, 511)
        nlt.set_ylim(0, 511)
        
        rat.set_xlim(0, 511)
        rat.set_ylim(0, 511)

        lte.xaxis.set_ticklabels([])
        lte.yaxis.set_ticklabels([])
        
        nlt.xaxis.set_ticklabels([])
        nlt.yaxis.set_ticklabels([])
        
        rat.xaxis.set_ticklabels([])
        rat.yaxis.set_ticklabels([])
        
        lte.xaxis.set_ticks([])
        lte.yaxis.set_ticks([])
    
        nlt.xaxis.set_ticks([])
        nlt.yaxis.set_ticks([])
        
        rat.xaxis.set_ticks([])
        rat.yaxis.set_ticks([])
    
        lte_img = lte.imshow(Is_lte[i - 1, :, :, j], cmap = colormap, vmin = imin, vmax = imax, norm = colors.LogNorm(), aspect = 'auto')
        nlt_img = nlt.imshow(Is_nlt[i - 1, :, :, j], cmap = colormap, vmin = imin, vmax = imax, norm = colors.LogNorm(), aspect = 'auto')
        rat_img = rat.imshow(Is_rat[i - 1, :, :],    cmap = colormap, vmin = rmin, vmax = rmax, norm = colors.LogNorm(), aspect = 'auto')
    
        int_cb = Colorbar(ax = int_cbar, mappable = lte_img, orientation = 'vertical', ticklocation = 'left')
        rat_cb = Colorbar(ax = rat_cbar, mappable = rat_img, orientation = 'vertical', ticklocation = 'left')
        
        lte.set_title(r'$\mu = $' + mus[i - 1], fontsize = 5)
    
        if i == 5:
            
            lte.set_ylabel('LTE')
            nlt.set_ylabel('NLTE')
            
            lte.yaxis.set_label_position("right")
            nlt.yaxis.set_label_position("right")
        
    int_cb.set_label('SSD intensity, cgs', labelpad = 5)
    rat_cb.set_label(r'$I_\mathrm{NLTE} / I_\mathrm{LTE}$', labelpad = 5)
    
    spec = plt.subplot(gs[3, :])

    spec.plot(w, Is_lte_m[0, :], color = 'k', linewidth = 0.5, label = 'LTE')
    spec.plot(w, Is_nlt_m[0, :], color = 'r', linewidth = 0.5, label = 'NLTE')
    
    spec.axvline(x = w[j], linestyle = '--', linewidth = 0.5)

    spec.set_yscale('log')

    spec.set_xlim(180, 300)

    spec.set_xlabel('Wavelength, nm')
    spec.set_ylabel('SSD intensity, cgs')

    spec.xaxis.set_minor_locator(AutoMinorLocator(10))
    
    leg = spec.legend(framealpha = 1, loc = 4, handletextpad = 1, prop = {'size': 8.5})

#    plt.show()

    plt.savefig('./fig/img/' + str(j) + '.pdf', bbox_inches = 'tight')
    
os.system('pdftk ' + pdfs + ' output ./fig/img/all.pdf')

100%|██████████| 1/1 [00:26<00:00, 26.79s/it]


0

In [7]:
#I_nlte, I_lte and I_nlte / I_lte for 300G, plus specta

import matplotlib.pyplot as plt

import matplotlib.gridspec as gridspec
from matplotlib.colorbar import Colorbar

from matplotlib.ticker import AutoMinorLocator
from matplotlib.ticker import MultipleLocator

from matplotlib import colors

import os

from tqdm import tqdm

plt.rcParams.update({'font.size': 8})
plt.rcParams["font.family"] = 'Times New Roman'

I3_rat = np.zeros((5, Nx, Ny))

w = np.arange(180, 300, 1) + 0.5

#j = 20

pdfs = ''

#for j in tqdm(range(Nz)):
for j in tqdm(range(19, 20)):
    
    pdfs += './fig/img/' + str(j) + '.pdf '

    plt.close('all')

    fig = plt.figure(dpi = 300)

    fig.set_size_inches(5, 6, forward = True)

    fig.tight_layout()

    gs = gridspec.GridSpec(4, 6, width_ratios = [0.05, 1, 0.8, 0.6, 0.4, 0.2], hspace = 0.04, wspace = 0.06)

    int_cbar = plt.subplot(gs[:2, 0])
    rat_cbar = plt.subplot(gs[2, 0])

    colormap = 'plasma'

    mus = ['1.0', '0.8', '0.6', '0.4', '0.2']
    
    for i in range(1, 6):

        I3_rat[i - 1, :, :] = I3_nlt[i - 1, :, :, j] / I3_lte[i - 1, :, :, j]
        
    rmin = np.percentile(I3_rat, 5)
    rmax = np.percentile(I3_rat, 95)
    
    lmin = np.percentile(I3_lte[:, :, :, j], 5)
    lmax = np.percentile(I3_lte[:, :, :, j], 95)
    
    nmin = np.percentile(I3_nlt[:, :, :, j], 5)
    nmax = np.percentile(I3_nlt[:, :, :, j], 95)
    
    imin = min(lmin, nmin)
    imax = max(lmax, nmax)

    for i in range(1, 6):

        lte = plt.subplot(gs[0, i])
        nlt = plt.subplot(gs[1, i])
        
        rat = plt.subplot(gs[2, i])

        lte.set_xlim(0, 511)
        lte.set_ylim(0, 511)

        nlt.set_xlim(0, 511)
        nlt.set_ylim(0, 511)
        
        rat.set_xlim(0, 511)
        rat.set_ylim(0, 511)

        lte.xaxis.set_ticklabels([])
        lte.yaxis.set_ticklabels([])
        
        nlt.xaxis.set_ticklabels([])
        nlt.yaxis.set_ticklabels([])
        
        rat.xaxis.set_ticklabels([])
        rat.yaxis.set_ticklabels([])
        
        lte.xaxis.set_ticks([])
        lte.yaxis.set_ticks([])
    
        nlt.xaxis.set_ticks([])
        nlt.yaxis.set_ticks([])
        
        rat.xaxis.set_ticks([])
        rat.yaxis.set_ticks([])
    
        lte_img = lte.imshow(I3_lte[i - 1, :, :, j], cmap = colormap, vmin = imin, vmax = imax, norm = colors.LogNorm(), aspect = 'auto')
        nlt_img = nlt.imshow(I3_nlt[i - 1, :, :, j], cmap = colormap, vmin = imin, vmax = imax, norm = colors.LogNorm(), aspect = 'auto')
        rat_img = rat.imshow(I3_rat[i - 1, :, :],    cmap = colormap, vmin = rmin, vmax = rmax, norm = colors.LogNorm(), aspect = 'auto')
    
        int_cb = Colorbar(ax = int_cbar, mappable = lte_img, orientation = 'vertical', ticklocation = 'left')
        rat_cb = Colorbar(ax = rat_cbar, mappable = rat_img, orientation = 'vertical', ticklocation = 'left')
        
        lte.set_title(r'$\mu = $' + mus[i - 1], fontsize = 5)
    
        if i == 5:
            
            lte.set_ylabel('LTE')
            nlt.set_ylabel('NLTE')
            
            lte.yaxis.set_label_position("right")
            nlt.yaxis.set_label_position("right")
        
    int_cb.set_label('300G intensity, cgs', labelpad = 5)
    rat_cb.set_label(r'$I_\mathrm{NLTE} / I_\mathrm{LTE}$', labelpad = 5)
    
    spec = plt.subplot(gs[3, :])

    spec.plot(w, I3_lte_m[0, :], color = 'k', linewidth = 0.5, label = 'LTE')
    spec.plot(w, I3_nlt_m[0, :], color = 'r', linewidth = 0.5, label = 'NLTE')
    
    spec.axvline(x = w[j], linestyle = '--', linewidth = 0.5)

    spec.set_yscale('log')

    spec.set_xlim(180, 300)

    spec.set_xlabel('Wavelength, nm')
    spec.set_ylabel('300G intensity, cgs')

    spec.xaxis.set_minor_locator(AutoMinorLocator(10))
    
    leg = spec.legend(framealpha = 1, loc = 4, handletextpad = 1, prop = {'size': 8.5})

#    plt.show()

    plt.savefig('./fig/img/' + str(j) + '.pdf', bbox_inches = 'tight')
    
os.system('pdftk ' + pdfs + ' output ./fig/img/all.pdf')

100%|██████████| 1/1 [00:33<00:00, 33.15s/it]


0

In [3]:
rmin = np.load('min_max_norm.npz')['rmin']
rmax = np.load('min_max_norm.npz')['rmax']

imin = np.load('min_max_norm.npz')['imin']
imax = np.load('min_max_norm.npz')['imax']

print('rmin = ', rmin, '; rmax = ', rmax, '; imin = ', imin, '; imax = ', imax)

rmin =  0.10964333383130302 ; rmax =  20.95521720193392 ; imin =  0.022549299383105494 ; imax =  20.346921648489662


In [5]:
#I_nlte, I_lte and I_nlte / I_lte for SSD and 300G, plus specta

import matplotlib.pyplot as plt

import matplotlib.gridspec as gridspec
from matplotlib.colorbar import Colorbar

from matplotlib.ticker import AutoMinorLocator
from matplotlib.ticker import MultipleLocator

from matplotlib import colors

import os

from tqdm import tqdm

props = dict(boxstyle = 'round', facecolor = 'wheat', alpha = 0.5)

plt.rcParams.update({'font.size': 8})
plt.rcParams["font.family"] = 'Times New Roman'

Is_rat = np.zeros((5, Nx, Ny))
I3_rat = np.zeros((5, Nx, Ny))

w = np.arange(180, 300, 1) + 0.5

#j = 20

pdfs = ''

for j in tqdm(range(Nz)):
#for j in tqdm(range(19, 20)):
    
    pdfs += './fig/img/' + str(j) + '.pdf '

    plt.close('all')

    fig = plt.figure(dpi = 300)

    fig.set_size_inches(10, 6, forward = True)

    fig.tight_layout()

    gs = gridspec.GridSpec(4, 11, width_ratios = [0.05, 1, 0.8, 0.6, 0.4, 0.2, 1, 0.8, 0.6, 0.4, 0.2], hspace = 0.03, wspace = 0.05)

    int_cbar = plt.subplot(gs[:2, 0])
    rat_cbar = plt.subplot(gs[2, 0])

    colormap = 'plasma'

    mus = ['1.0', '0.8', '0.6', '0.4', '0.2']
    
    for i in range(1, 6):

        Is_rat[i - 1, :, :] = Is_nlt[i - 1, :, :, j] / Is_lte[i - 1, :, :, j]
        I3_rat[i - 1, :, :] = I3_nlt[i - 1, :, :, j] / I3_lte[i - 1, :, :, j]
        
    rmin = min(np.percentile(Is_rat, 5), np.percentile(I3_rat, 5))
    rmax = max(np.percentile(Is_rat, 95), np.percentile(I3_rat, 95))
    
    lmin = min(np.percentile(Is_lte[:, :, :, j], 5), np.percentile(I3_lte[:, :, :, j], 5))
    lmax = max(np.percentile(Is_lte[:, :, :, j], 95), np.percentile(I3_lte[:, :, :, j], 95))
    
    nmin = min(np.percentile(Is_nlt[:, :, :, j], 5), np.percentile(I3_nlt[:, :, :, j], 5))
    nmax = max(np.percentile(Is_nlt[:, :, :, j], 95), np.percentile(I3_nlt[:, :, :, j], 95))
    
    imin = min(lmin, nmin)
    imax = max(lmax, nmax)

    for i in range(1, 6):

#------------------------------------------
# settings for SSD

        ssd_lte = plt.subplot(gs[0, i])
        ssd_nlt = plt.subplot(gs[1, i])
        
        ssd_rat = plt.subplot(gs[2, i])

        ssd_lte.set_xlim(0, 511)
        ssd_lte.set_ylim(0, 511)

        ssd_nlt.set_xlim(0, 511)
        ssd_nlt.set_ylim(0, 511)
        
        ssd_rat.set_xlim(0, 511)
        ssd_rat.set_ylim(0, 511)

        ssd_lte.xaxis.set_ticklabels([])
        ssd_lte.yaxis.set_ticklabels([])
        
        ssd_nlt.xaxis.set_ticklabels([])
        ssd_nlt.yaxis.set_ticklabels([])
        
        ssd_rat.xaxis.set_ticklabels([])
        ssd_rat.yaxis.set_ticklabels([])
        
        ssd_lte.xaxis.set_ticks([])
        ssd_lte.yaxis.set_ticks([])
    
        ssd_nlt.xaxis.set_ticks([])
        ssd_nlt.yaxis.set_ticks([])
        
        ssd_rat.xaxis.set_ticks([])
        ssd_rat.yaxis.set_ticks([])

#------------------------------------------
# settings for 300G

        mag_lte = plt.subplot(gs[0, 5 + i])
        mag_nlt = plt.subplot(gs[1, 5 + i])
        
        mag_rat = plt.subplot(gs[2, 5 + i])

        mag_lte.set_xlim(0, 511)
        mag_lte.set_ylim(0, 511)

        mag_nlt.set_xlim(0, 511)
        mag_nlt.set_ylim(0, 511)
        
        mag_rat.set_xlim(0, 511)
        mag_rat.set_ylim(0, 511)

        mag_lte.xaxis.set_ticklabels([])
        mag_lte.yaxis.set_ticklabels([])
        
        mag_nlt.xaxis.set_ticklabels([])
        mag_nlt.yaxis.set_ticklabels([])
        
        mag_rat.xaxis.set_ticklabels([])
        mag_rat.yaxis.set_ticklabels([])
        
        mag_lte.xaxis.set_ticks([])
        mag_lte.yaxis.set_ticks([])
    
        mag_nlt.xaxis.set_ticks([])
        mag_nlt.yaxis.set_ticks([])
        
        mag_rat.xaxis.set_ticks([])
        mag_rat.yaxis.set_ticks([])
        
#----------------------------------------------
    
        ssd_lte_img = ssd_lte.imshow(Is_lte[i - 1, :, :, j], cmap = colormap, vmin = imin, vmax = imax, norm = colors.LogNorm(), aspect = 'auto')
        ssd_nlt_img = ssd_nlt.imshow(Is_nlt[i - 1, :, :, j], cmap = colormap, vmin = imin, vmax = imax, norm = colors.LogNorm(), aspect = 'auto')
        ssd_rat_img = ssd_rat.imshow(Is_rat[i - 1, :, :],    cmap = colormap, vmin = rmin, vmax = rmax, norm = colors.LogNorm(), aspect = 'auto')
        
        mag_lte_img = mag_lte.imshow(I3_lte[i - 1, :, :, j], cmap = colormap, vmin = imin, vmax = imax, norm = colors.LogNorm(), aspect = 'auto')
        mag_nlt_img = mag_nlt.imshow(I3_nlt[i - 1, :, :, j], cmap = colormap, vmin = imin, vmax = imax, norm = colors.LogNorm(), aspect = 'auto')
        mag_rat_img = mag_rat.imshow(I3_rat[i - 1, :, :],    cmap = colormap, vmin = rmin, vmax = rmax, norm = colors.LogNorm(), aspect = 'auto')
    
        int_cb = Colorbar(ax = int_cbar, mappable = ssd_lte_img, orientation = 'vertical', ticklocation = 'left')
        rat_cb = Colorbar(ax = rat_cbar, mappable = ssd_rat_img, orientation = 'vertical', ticklocation = 'left')
        
        ssd_lte.set_title(r'$\mu = $' + mus[i - 1], fontsize = 5)
        mag_lte.set_title(r'$\mu = $' + mus[i - 1], fontsize = 5)
    
        if i == 5:
            
            mag_lte.set_ylabel('LTE')
            mag_nlt.set_ylabel('NLTE')
            
            mag_lte.yaxis.set_label_position("right")
            mag_nlt.yaxis.set_label_position("right")
        
#    int_cb.set_label('Intensity, cgs', labelpad = 5)
    int_cb.set_label('Normalized intensity', labelpad = 5)
#    rat_cb.set_label(r'$I_\mathrm{NLTE} / I_\mathrm{LTE}$', labelpad = 5)
    rat_cb.set_label(r'$I^\mathrm{n}_\mathrm{NLTE} / I^\mathrm{n}_\mathrm{LTE}$', labelpad = 5)
    
    ssd_spec = plt.subplot(gs[3, : 6])

    ssd_spec.plot(w, Is_lte_m[0, :], color = 'k', linewidth = 0.5, label = 'LTE')
    ssd_spec.plot(w, Is_nlt_m[0, :], color = 'r', linewidth = 0.5, label = 'NLTE')
    
    ssd_spec.axvline(x = w[j], linestyle = '--', linewidth = 0.5)

    ssd_spec.set_yscale('log')

    ssd_spec.set_xlim(180, 300)

    ssd_spec.set_xlabel('Wavelength, nm')
    ssd_spec.set_ylabel('Intensity, cgs')

    ssd_spec.xaxis.set_minor_locator(AutoMinorLocator(10))
    
    ssd_spec.xaxis.set_ticklabels(['180', '200', '220', '240', '260', '280', ' '])
    
#    ssd_leg = ssd_spec.legend(framealpha = 1, loc = 4, handletextpad = 1, prop = {'size': 8.5})

    ssd_spec.text(183, 3.5e-6, 'SSD', fontsize = 10, verticalalignment = 'top', bbox = props)
    
    mag_spec = plt.subplot(gs[3, 6 :])

    mag_spec.plot(w, I3_lte_m[0, :], color = 'k', linewidth = 0.5, label = 'LTE')
    mag_spec.plot(w, I3_nlt_m[0, :], color = 'r', linewidth = 0.5, label = 'NLTE')
    
    mag_spec.axvline(x = w[j], linestyle = '--', linewidth = 0.5)

    mag_spec.set_yscale('log')

    mag_spec.set_xlim(180, 300)
    
    mag_spec.set_ylim(ssd_spec.get_ylim())

    mag_spec.set_xlabel('Wavelength, nm')

    mag_spec.xaxis.set_minor_locator(AutoMinorLocator(10))
    
    mag_spec.xaxis.set_ticklabels([' ', '200', '220', '240', '260', '280', '300'])
    mag_spec.yaxis.set_ticklabels([])
    
    mag_spec.tick_params(axis = 'y', which = 'both', direction = 'in')
    
    mag_spec.text(183, 3.5e-6, '300G', fontsize = 10, verticalalignment = 'top', bbox = props)
    
    mag_leg = mag_spec.legend(framealpha = 1, loc = 4, handletextpad = 1, prop = {'size': 8.5})

    plt.savefig('./fig/img_plus_spec/' + str(j) + '.pdf', bbox_inches = 'tight')
    
os.system('pdftk ' + pdfs + ' output ./fig/img_plus_spec/all.pdf')

100%|██████████| 120/120 [32:05<00:00, 34.29s/it]


0

In [14]:
#I_nlte, I_lte and I_nlte / I_lte for SSD and 300G

import numpy as np

import matplotlib.pyplot as plt

import matplotlib.gridspec as gridspec
from matplotlib.colorbar import Colorbar

from matplotlib.ticker import AutoMinorLocator
from matplotlib.ticker import MultipleLocator

import matplotlib.colors as colors

from  matplotlib.colors import LogNorm

import os

from tqdm import tqdm

class MidPointLogNorm(LogNorm):
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        LogNorm.__init__(self,vmin=vmin, vmax=vmax, clip=clip)
        self.midpoint=midpoint
    def __call__(self, value, clip=None):
        # I'm ignoring masked values and all kinds of edge cases to make a
        # simple example...
        x, y = [np.log(self.vmin), np.log(self.midpoint), np.log(self.vmax)], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(np.log(value), x, y))

props = dict(boxstyle = 'round', facecolor = 'wheat', alpha = 0.5)

plt.rcParams.update({'font.size': 8})
plt.rcParams["font.family"] = 'Times New Roman'

Is_rat = np.zeros((5, Nx, Ny))
I3_rat = np.zeros((5, Nx, Ny))

w = np.arange(180, 300, 1) + 0.5

colormap1 = 'plasma'
colormap2 = 'RdBu_r'

mus = ['1.0', '0.8', '0.6', '0.4', '0.2']

pdfs = ''

for j in tqdm([7, 13, 50, 82, 84]):

#for j in tqdm(range(Nz)):
#for j in tqdm(range(7, 8)):
#for j in tqdm(range(13, 14)):
#for j in tqdm(range(50, 51)):
#for j in tqdm(range(82, 83)):
#for j in tqdm(range(84, 85)):
    
    pdfs += './fig/img/' + str(j) + '.pdf '

    plt.close('all')

    fig = plt.figure(dpi = 300)

    fig.set_size_inches(10, 4.5, forward = True)

    fig.tight_layout()

    gs = gridspec.GridSpec(3, 11, width_ratios = [0.05, 1, 0.8, 0.6, 0.4, 0.2, 1, 0.8, 0.6, 0.4, 0.2], hspace = 0.03, wspace = 0.05)

    int_cbar = plt.subplot(gs[:2, 0])
    rat_cbar = plt.subplot(gs[2, 0])
    
    for i in range(1, 6):

        Is_rat[i - 1, :, :] = Is_nlt[i - 1, :, :, j] / Is_lte[i - 1, :, :, j]
        I3_rat[i - 1, :, :] = I3_nlt[i - 1, :, :, j] / I3_lte[i - 1, :, :, j]
        
    rmin = min(np.percentile(Is_rat, 5), np.percentile(I3_rat, 5))
    rmax = max(np.percentile(Is_rat, 95), np.percentile(I3_rat, 95))
    
    lmin = min(np.percentile(Is_lte[:, :, :, j], 5), np.percentile(I3_lte[:, :, :, j], 5))
    lmax = max(np.percentile(Is_lte[:, :, :, j], 95), np.percentile(I3_lte[:, :, :, j], 95))
    
    nmin = min(np.percentile(Is_nlt[:, :, :, j], 5), np.percentile(I3_nlt[:, :, :, j], 5))
    nmax = max(np.percentile(Is_nlt[:, :, :, j], 95), np.percentile(I3_nlt[:, :, :, j], 95))
    
    imin = min(lmin, nmin)
    imax = max(lmax, nmax)

    for i in range(1, 6):

#------------------------------------------
# settings for SSD

        ssd_lte = plt.subplot(gs[0, i])
        ssd_nlt = plt.subplot(gs[1, i])
        
        ssd_rat = plt.subplot(gs[2, i])
        
        if i == 2: ssd_lte.text(300, 600, 'SSD', fontsize = 10, verticalalignment = 'top', bbox = props)
        if i == 4: ssd_lte.text(400, 600, r'$\lambda = $' + str(w[j]) + ' nm', fontsize = 10, verticalalignment = 'top', bbox = props)

        ssd_lte.set_xlim(0, 511)
        ssd_lte.set_ylim(0, 511)

        ssd_nlt.set_xlim(0, 511)
        ssd_nlt.set_ylim(0, 511)
        
        ssd_rat.set_xlim(0, 511)
        ssd_rat.set_ylim(0, 511)

        ssd_lte.xaxis.set_ticklabels([])
        ssd_lte.yaxis.set_ticklabels([])
        
        ssd_nlt.xaxis.set_ticklabels([])
        ssd_nlt.yaxis.set_ticklabels([])
        
        ssd_rat.xaxis.set_ticklabels([])
        ssd_rat.yaxis.set_ticklabels([])
        
        ssd_lte.xaxis.set_ticks([])
        ssd_lte.yaxis.set_ticks([])
    
        ssd_nlt.xaxis.set_ticks([])
        ssd_nlt.yaxis.set_ticks([])
        
        ssd_rat.xaxis.set_ticks([])
        ssd_rat.yaxis.set_ticks([])

#------------------------------------------
# settings for 300G

        mag_lte = plt.subplot(gs[0, 5 + i])
        mag_nlt = plt.subplot(gs[1, 5 + i])
        
        mag_rat = plt.subplot(gs[2, 5 + i])
        
        if i == 2: mag_lte.text(200, 600, '300G', fontsize = 10, verticalalignment = 'top', bbox = props)

        mag_lte.set_xlim(0, 511)
        mag_lte.set_ylim(0, 511)

        mag_nlt.set_xlim(0, 511)
        mag_nlt.set_ylim(0, 511)
        
        mag_rat.set_xlim(0, 511)
        mag_rat.set_ylim(0, 511)

        mag_lte.xaxis.set_ticklabels([])
        mag_lte.yaxis.set_ticklabels([])
        
        mag_nlt.xaxis.set_ticklabels([])
        mag_nlt.yaxis.set_ticklabels([])
        
        mag_rat.xaxis.set_ticklabels([])
        mag_rat.yaxis.set_ticklabels([])
        
        mag_lte.xaxis.set_ticks([])
        mag_lte.yaxis.set_ticks([])
    
        mag_nlt.xaxis.set_ticks([])
        mag_nlt.yaxis.set_ticks([])
        
        mag_rat.xaxis.set_ticks([])
        mag_rat.yaxis.set_ticks([])
        
#----------------------------------------------
        
        lonorm = colors.LogNorm(vmin = imin, vmax = imax)
        
        if j == 7: mpnorm = MidPointLogNorm(vmin = rmin, vmax = rmax, midpoint = 1)
        if j != 7: mpnorm = colors.TwoSlopeNorm(vmin = rmin, vmax = rmax, vcenter = 1)
    
        ssd_lte_img = ssd_lte.imshow(Is_lte[i - 1, :, :, j], cmap = colormap1, norm = lonorm, aspect = 'auto')
        ssd_nlt_img = ssd_nlt.imshow(Is_nlt[i - 1, :, :, j], cmap = colormap1, norm = lonorm, aspect = 'auto')
        ssd_rat_img = ssd_rat.imshow(Is_rat[i - 1, :, :],    cmap = colormap2, norm = mpnorm, aspect = 'auto')
        
        mag_lte_img = mag_lte.imshow(I3_lte[i - 1, :, :, j], cmap = colormap1, norm = lonorm, aspect = 'auto')
        mag_nlt_img = mag_nlt.imshow(I3_nlt[i - 1, :, :, j], cmap = colormap1, norm = lonorm, aspect = 'auto')
        mag_rat_img = mag_rat.imshow(I3_rat[i - 1, :, :],    cmap = colormap2, norm = mpnorm, aspect = 'auto')
    
        int_cb = Colorbar(ax = int_cbar, mappable = ssd_lte_img, orientation = 'vertical', ticklocation = 'left')
        rat_cb = Colorbar(ax = rat_cbar, mappable = ssd_rat_img, orientation = 'vertical', ticklocation = 'left')
        
        ssd_rat.set_xlabel(r'$\mu = $' + mus[i - 1], fontsize = 5)
        mag_rat.set_xlabel(r'$\mu = $' + mus[i - 1], fontsize = 5)
    
        if i == 5:
            
            mag_lte.set_ylabel('LTE')
            mag_nlt.set_ylabel('NLTE')
            
            mag_lte.yaxis.set_label_position("right")
            mag_nlt.yaxis.set_label_position("right")
        
    int_cb.set_label('Intensity, cgs', labelpad = 5)
#    int_cb.set_label('Normalized intensity', labelpad = 5)
    rat_cb.set_label(r'$I_\mathrm{NLTE} / I_\mathrm{LTE}$', labelpad = 5)
#    rat_cb.set_label(r'$I^\mathrm{n}_\mathrm{NLTE} / I^\mathrm{n}_\mathrm{LTE}$', labelpad = 5)
    
#    ssd_spec = plt.subplot(gs[3, : 6])

#    ssd_spec.plot(w, Is_lte_m[0, :], color = 'k', linewidth = 0.5, label = 'LTE')
#    ssd_spec.plot(w, Is_nlt_m[0, :], color = 'r', linewidth = 0.5, label = 'NLTE')
    
#    ssd_spec.axvline(x = w[j], linestyle = '--', linewidth = 0.5)

#    ssd_spec.set_yscale('log')

#    ssd_spec.set_xlim(180, 300)

#    ssd_spec.set_xlabel('Wavelength, nm')
#    ssd_spec.set_ylabel('Intensity, cgs')

#    ssd_spec.xaxis.set_minor_locator(AutoMinorLocator(10))
    
#    ssd_spec.xaxis.set_ticklabels(['180', '200', '220', '240', '260', '280', ' '])
    
#    ssd_leg = ssd_spec.legend(framealpha = 1, loc = 4, handletextpad = 1, prop = {'size': 8.5})

#    ssd_spec.text(183, 3.5e-6, 'SSD', fontsize = 10, verticalalignment = 'top', bbox = props)
    
#    mag_spec = plt.subplot(gs[3, 6 :])

#    mag_spec.plot(w, I3_lte_m[0, :], color = 'k', linewidth = 0.5, label = 'LTE')
#    mag_spec.plot(w, I3_nlt_m[0, :], color = 'r', linewidth = 0.5, label = 'NLTE')
    
#    mag_spec.axvline(x = w[j], linestyle = '--', linewidth = 0.5)

#    mag_spec.set_yscale('log')

#    mag_spec.set_xlim(180, 300)
    
#    mag_spec.set_ylim(ssd_spec.get_ylim())

#    mag_spec.set_xlabel('Wavelength, nm')

#    mag_spec.xaxis.set_minor_locator(AutoMinorLocator(10))
    
#    mag_spec.xaxis.set_ticklabels([' ', '200', '220', '240', '260', '280', '300'])
#    mag_spec.yaxis.set_ticklabels([])
    
#    mag_spec.tick_params(axis = 'y', which = 'both', direction = 'in')
    
#    mag_spec.text(183, 3.5e-6, '300G', fontsize = 10, verticalalignment = 'top', bbox = props)
    
#    mag_leg = mag_spec.legend(framealpha = 1, loc = 4, handletextpad = 1, prop = {'size': 8.5})

    plt.savefig('./fig/img/img_' + str(j) + '.pdf', bbox_inches = 'tight')
    
#os.system('pdftk ' + pdfs + ' output ./fig/img/all.pdf')


100%|██████████| 5/5 [00:50<00:00, 11.02s/it]
